## How to get csv files for analysis below
- Make sure your shells pwd is this directory
- Connect to an analytics database
- Run csv export (adjust the timestamp filter first)
  ```psql
  \copy (select * from areaoccupancy a join metadata m on m.id = a.metadata_id where a.occupancy_time > '2025-12-10' order by a.occupancy_time) to 'tsp-area.csv' with csv header
  \copy (select * from linecrossing l join metadata m on m.id = l.metadata_id where l.crossing_time > '2025-12-10' order by l.crossing_time) to 'tsp-line.csv' with csv header
  ```

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# Whenever you change the code in imported modules, just rerun this cell, autoreload makes restarting the kernel unnecessary

from common import read_area_csv, read_line_csv

In [ ]:
from datetime import datetime

import pandas as pd


def simple_counting(line_df: pd.DataFrame) -> pd.DataFrame:
    results = []
    count = 0
    for crossing in line_df.itertuples():
        if crossing.direction == 'in':
            count += 1
        else:
            count -= 1
        results.append([crossing.crossing_time, count])
    return pd.DataFrame(columns=['flow_occupancy_time', 'count'], data=results)

def lower_bounded_counting(line_df: pd.DataFrame) -> pd.DataFrame:
    results = []
    count = 0
    for crossing in line_df.itertuples():
        if crossing.direction == 'in':
            count += 1
        else:
            count = max(0, count - 1)
        results.append([crossing.crossing_time, count])
    return pd.DataFrame(columns=['flow_occupancy_time', 'count'], data=results)

def bounded_counting(line_df: pd.DataFrame, upper_bound: int) -> pd.DataFrame:
    results = []
    count = 0
    for crossing in line_df.itertuples():
        if crossing.direction == 'in':
            count = min(upper_bound, count + 1)
        else:
            count = max(0, count - 1)
        results.append([crossing.crossing_time, count])
    return pd.DataFrame(columns=['flow_occupancy_time', 'count'], data=results)

In [ ]:
MECKAUER_AREAS = [
    'parking-meckauer-west01',
    'parking-meckauer-west02',
    'parking-meckauer-west03',
    'parking-meckauer-center',
    'parking-meckauer-center02',
    'parking-meckauer-ost01',
    'parking-meckauer-ost02',
]
tsp_staging_area_df = read_area_csv('tsp-area.csv', MECKAUER_AREAS)
tsp_dev_area_df = read_area_csv('dev-area.csv', MECKAUER_AREAS)

tsp_staging_line_df = read_line_csv('tsp-line.csv', 'flow-meckauer-main')
tsp_staging_flow_df_simple = simple_counting(tsp_staging_line_df)
tsp_staging_flow_df_lower_bounded = lower_bounded_counting(tsp_staging_line_df)
tsp_staging_flow_df_bounded = bounded_counting(tsp_staging_line_df, upper_bound=75)

In [ ]:
the_line_df = read_line_csv('tsp-line.csv', 'theater-parking-flow', datetime.fromisoformat('2026-01-06T00:00:00Z'))
the_line_df_simple = simple_counting(the_line_df)
the_line_df_lower_bounded = lower_bounded_counting(the_line_df)
the_line_df_bounded = bounded_counting(the_line_df, upper_bound=250)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# fig.add_trace(
#     go.Scattergl(
#         x=staging_area_df.index, 
#         y=staging_area_df['total_count'],
#         mode='lines',        # or "lines+markers"
#         name='area_occupancy (ground truth)'
#     ),
# )

def add_algo_result(df: pd.DataFrame, name: str):
    fig.add_trace(
        go.Scattergl(
            x=df['flow_occupancy_time'], 
            y=df['count'],
            mode='lines',        # or "lines+markers"
            name=name
        ),
    )


# add_algo_result(staging_flow_df_simple, 'simple counting')
# add_algo_result(staging_flow_df_lower_bounded, 'lower bounded counting')
# add_algo_result(staging_flow_df_bounded, 'bounded counting (UB=75)')
add_algo_result(the_line_df_simple, 'the-line simple counting')
add_algo_result(the_line_df_lower_bounded, 'the-line lower bounded counting')
add_algo_result(the_line_df_bounded, 'the-line bounded counting (UB=250)')

fig.update_layout(
    title='Count over Time',
    xaxis_title='time',
    yaxis_title='total_count',
)

fig.show()